In [3]:
import tensorflow as tf

from classes.models import LeakyRNNCell, LeakyRNN
from classes.datagen import genFactory

2025-06-02 19:20:06.225035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-02 19:20:06.237911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-02 19:20:06.241805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 19:20:06.252929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-02 19:20:07.019338: W tensorflow/compiler/tf2

In [7]:
task = 'just_short_match'
params = {
    'units': 128,
    'activation': 'tanh',
    'weights_regularizer_coef': 1e-3,
    'activity_regularizer_coef': 1e-3,
    'learning_rate': 1e-5,
    'noise_level': 0.3,
    'input_noise': 0.15,
    'gamma': 0.2,
    'input_blocks_per_epoch': 1,
    'batch_size': 2
}

In [5]:
activity_regularizer = tf.keras.regularizers.L2(params['activity_regularizer_coef'])
recurrent_regularizer = tf.keras.regularizers.L2(params['activity_regularizer_coef'])
kernel_regularizer = tf.keras.regularizers.L2(params['weights_regularizer_coef'])

optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

model = tf.keras.models.Sequential([
    LeakyRNN(params['units'], activation=params['activation'], gamma=params['gamma'],
             return_sequences=True, activity_regularizer=activity_regularizer,
             recurrent_regularizer=recurrent_regularizer, kernel_regularizer=kernel_regularizer),
    tf.keras.layers.Dense(
        units=1, activity_regularizer=activity_regularizer,
        kernel_regularizer=kernel_regularizer)
])
model.compile(loss=tf.keras.losses.MeanSquaredError(),
             optimizer=optimizer, metrics=[tf.keras.metrics.MeanSquaredError()])  

I0000 00:00:1748892011.500563 3822335 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-02 19:20:11.502852: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [20]:
traingen = genFactory.create(task, params['input_noise'], params['batch_size'], params['input_blocks_per_epoch'])
validation_data = traingen.generate_trials(10)
formatted_validation = traingen.__class__.format_validation(validation_data)

2025-06-02 19:35:07.743651: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [41]:
history = model.fit(traingen, validation_data=formatted_validation, epochs=3)

Epoch 1/3
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - loss: 0.1576 - mean_squared_error: 0.0368 - val_loss: 0.2059 - val_mean_squared_error: 0.0302
Epoch 2/3
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - loss: 0.1383 - mean_squared_error: 0.0201 - val_loss: 0.1910 - val_mean_squared_error: 0.0301
Epoch 3/3
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - loss: 0.1520 - mean_squared_error: 0.0339 - val_loss: 0.1959 - val_mean_squared_error: 0.0288


In [28]:
history.history['val_mean_squared_error']
formatted_validation[0].shape

(1, 8000, 2)